# XGBoost regression (single-node)

<img src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/xgboost.png" width="300">

This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll load data into a `pandas` data frame and use XGBoost to answer this question

> based on characteristics that can be known at the beginning of a trip, what tip will this trip earn (as a % of the total fare)?

**NOTE:** This notebook has some cells that can take 3-10 minutes to run. Consider opening [this Dask + XGBoost notebook](./xgboost-dask.ipynb) and running that while you're waiting for cells in this notebook to complete.

<hr>

## Load data

This example is designed to run quickly with small, relatively inexpensive resources. So let's just load a single month of taxi data for training.

In [ ]:
import pandas as pd

taxi = pd.read_csv(
    "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
).sample(frac=0.3, replace=False)

The code below computes the size of this dataset in memory. One month is about 7.6 million rows and 1.5 GB.

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {round(taxi.memory_usage(deep=True).sum() / 1e9, 2)} GB")

You can examine the structure of the data with `pandas` commands:

`.head()` = view the first few rows

In [ ]:
taxi.head()

`.dtypes` = list all the columns and the type of data in them

In [ ]:
taxi.dtypes

<hr>

## Prep for Training

Before training a model, we need to transform this dataset into a format that's better-suited to the research question. The function below does that with `pandas` operations.

<details><summary>(click here to learn why data scientists do this)</summary>

**Compute the Target**

The raw data don't contain a column that cleanly describes the tip as a percentage of the total fare. So we need to add one!

**Add Features**

Giving a machine learning model a richer description of each training observation improves its ability to describe the relationship between those observations' characteristics and the target. These characteristics are called "features".

For example, instead of giving a model a raw timestamp, it can be valuable to provide multiple derived characteristics like hour of the day and day of the week. It's plausible, for example, that weekend rides might have a different distribution of tips because they tend to be for leisure, where weekday rides might be mostly people travelling for work.

**Remove Unused Features**

If the goal is to produce a model that could predict the tip for a ride, then characteristics that can only be known AFTER the tip have to be excluded. For example, you can't know the dropoff time or the type of payment until a ride has concluded.

Such features should be dropped before training.
    
</details>

In [ ]:
def prep_df(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    """
    Prepare a raw taxi dataframe for training.
        * computes the target ('tip_fraction')
        * adds features
        * removes unused features
    """
    numeric_feat = [
        "pickup_weekday",
        "pickup_weekofyear",
        "pickup_hour",
        "pickup_week_hour",
        "pickup_minute",
        "passenger_count",
    ]
    categorical_feat = [
        "PULocationID",
        "DOLocationID",
    ]
    features = numeric_feat + categorical_feat
    df = df.copy()
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df[target_col] = df.tip_amount / df.fare_amount

    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_weekofyear"] = df.tpep_pickup_datetime.dt.isocalendar().week
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_week_hour"] = (df.pickup_weekday * 24) + df.pickup_hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [target_col]].astype(float).fillna(-1)

    return df

Run the code below to get a new data frame, `taxi_train`, that can be used directly for model training.

In [ ]:
target_col = "tip_fraction"
taxi_train = prep_df(taxi, target_col)

`taxi_train` is a `pandas` dataframe that will be passed in to a machine learning model. Before going further, check the first few rows of the dataset to make sure that the features look reasonable.

In [ ]:
taxi_train.head()

Now that the dataframe has been processed, check its size in memory again.

In [ ]:
print(
    f"Num rows: {len(taxi_train)}, Size: {round(taxi_train.memory_usage(deep=True).sum() / 1e9, 2)} GB"
)

As you can see above, removing unused columns dropped the size of the training data to 0.55 GB, about one third the size of the raw data.

<hr>

## Train a Model

Now that the data have been prepped, it's time to build a model!

For this task, we'll use the `XGBRegressor` from `xgboost`. If you've never used XGBoost or need a refresher, see [the XGBoost Python docs](https://xgboost.readthedocs.io/en/latest/python/index.html).

The code below initializes an `XGBRegressor` with the following parameter values.

* `objective = "reg:squarederror"`: solve a regression problem, and use mean squared error as the loss function
* `tree_method = "hist"`: use the "histogram" method of building trees. This method sacrifices a small amount of training accuracy for much faster training time.
* `learning_rate = 0.1`: controls how much each new tree contributes to the overall model. 0.1 is an arbitrary value.
* `max_depth = 5`: stop growing a tree once it contains a leaf node that is 5 levels below the root
* `n_estimators = 50`: create a model with at most 50 trees
* `n_jobs = multiprocessing.cpu_count()`: use all available cores on this machine to parallelize training
* `verbosity = 1`: print INFO-level logs and above

All other parameters use the defaults from [`XGBRegressor`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor).

<details><summary>(click here to learn why data scientists do this)</summary>

**Setting max_depth**
    
Tree-based models split the training data into smaller and smaller groups, to try to group together records with similar values of the target. A tree can be thought of as a collection of rules like `pickup_hour greater than 11` and `pickup_minute less than 31.0`. As you add more rules, those groups (called "leaf nodes") get smaller. In an extreme example, a model could create a tree with enough rules to place each record in the training data into its own group. That would probably take a lot of rules, and would be referred to as a "deep" tree.
    
Deep trees are problematic because their descriptions of the world are too specific to be useful on new data. Imagine training a classification model to predict whether or not visitors to a theme park will ride a particular rollercoaster. You could measure the time down to the millisecond that every guest's ticket is scanned at the entrance, and a model might learn a rule like *"if the guest has been to the park before and if the guest is older than 40 and younger than 41, and if the guest is staying at Hotel A and if the guest enters the park after 1:00:17.456 and if the guest enters the park earlier than 1:00:17.995, they will ride the rollercoaster"*. This is very very unlikely to ever match any future visitors, and if it does it's unlikely that this prediction will be very good unless you have some reason to believe that a visitor arriving at 1:00:18 instead of 1:00:17 really changes the probability that they'll ride that rollercoaster.
    
To prevent this situation (called "overfitting"), most tree-based machine learning algorithms accept parameters that control how deep the trees can get. `max_depth` is common, and says "don't create a rule more complex than this". In the example above, that rule has a depth of 7.
    
1. visiting the park
2. has been to the park before?
3. older than 40?
4. younger than 41?
5. staying at Hotel A?
6. entered the park after 1:00:17.456?
7. entered the park before 1:00:17.995?
    
Setting `max_depth = 5` would have prevented those weirdly-specific timing rules from ever being generated.
    
Choosing good values for this parameter is part art, part science, and is outside the scope of this tutorial.
    
</details>

In [ ]:
import xgboost
import multiprocessing

xgb_reg = xgboost.XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",
    learning_rate=0.1,
    max_depth=5,
    n_estimators=50,
    n_jobs=multiprocessing.cpu_count(),
    verbosity=1,
)

With the regressor created, fit it to some data! The code below uses `%%time` to print out a timing, so you can see how long it takes to train. This can be used to compare single-node, CPU `xgboost` to methods explored in other notebooks, or to test how changing some parameters changes the runtime for training.

**NOTE:** This will take a few minutes to run. Consider opening [this Dask + XGBoost notebook](./xgboost-dask.ipynb) and running that while you're waiting for this model to train.

In [ ]:
%%time

features = [c for c in taxi_train.columns if c != target_col]

_ = xgb_reg.fit(taxi_train[features], y=taxi_train[target_col])

<hr>

## Save model

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/xgboost.pkl", "wb") as f:
    cloudpickle.dump(xgb_reg, f)

<hr>

## Calculate metrics on test set

Machine learning training tries to create a model which can produce useful results on new data that it didn't see during training. To test how well we've accomplished that in this example, read in another month of taxi data.

In [ ]:
taxi_test = pd.read_csv(
    "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-02.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
).sample(frac=0.01, replace=False)

Before creating predictions on this new dataset, it has to be transformed in exactly the way that the original training data were prepared. Thankfully you've already wrapped that transformation logic in a function!

In [ ]:
taxi_test = prep_df(taxi_test, target_col=target_col)

`scikit-learn` comes with many functions for calculating metrics that describe how well a model's predictions match the actual values. For a complete list, see ["Metrics and scoring"](https://scikit-learn.org/stable/modules/model_evaluation.html) in the `sciki-learn` docs.

This tutorial uses the `mean_squared_error` to evaluate the model. This metric penalizes large errors more than small errors.

In [ ]:
from sklearn.metrics import mean_squared_error

preds = xgb_reg.predict(taxi_test[features])
mean_squared_error(taxi_test[target_col], preds, squared=False)

<hr>

## Next Steps

In this tutorial, you learned how to use `xgboost` to train a tree-based supervised learning model on a single machine.

Next, try [this xgboost + Dask notebook](./xgboost-dask.ipynb) to learn how to use Dask to train larger models and / or reduce training time.

<hr>